In [2]:
import json
import io
import zipfile

In [4]:
# Read in json text files of tweets
tweetsRdd = sc.textFile("combined/*")
tweetsRdd.first()

u'{"id":"tag:search.twitter.com,2005:632084524101451776","objectType":"activity","actor":{"objectType":"person","id":"id:twitter.com:34908698","link":"http://www.twitter.com/SABreakingNews","displayName":"SA Breaking News","postedTime":"2009-04-24T12:09:12.000Z","image":"https://pbs.twimg.com/profile_images/598069459857907712/Yai1uIrW_normal.jpg","summary":"All the latest breaking news from across South Africa in one stream.","links":[{"href":"http://www.sabreakingnews.co.za/","rel":"me"}],"friendsCount":721,"followersCount":473761,"listedCount":1605,"statusesCount":262410,"twitterTimeZone":"Pretoria","verified":false,"utcOffset":"7200","preferredUsername":"SABreakingNews","languages":["en"],"location":{"objectType":"place","displayName":"South Africa"},"favoritesCount":14},"verb":"post","postedTime":"2015-08-14T07:01:08.000Z","generator":{"displayName":"Hootsuite","link":"http://www.hootsuite.com"},"provider":{"objectType":"service","displayName":"Twitter","link":"http://www.twitter.c

In [5]:
jsondict = tweetsRdd.map(lambda x: json.loads(x))

In [6]:
jsondict.count()

402507

In [7]:
tweetIDMapRDD = jsondict.map(lambda x:  x['id'].split(":")[-1])

In [52]:
tweetIDMapRDD.take(2)

[u'632084524101451776', u'632087068471087104']

In [8]:

jsondict.first()['id']

u'tag:search.twitter.com,2005:632084524101451776'

In [ ]:
getMediaUrl(jsondict.first())

In [ ]:
coordRdd = jsondict.flatMap(lambda x: getCoordinates(x))

In [ ]:
coordRdd.collect()

In [15]:
mediaRdd = jsondict.flatMap(lambda x: getMediaUrl(x))

In [ ]:
posturlS3(mediaRdd.first())

In [ ]:
print "hello"

In [ ]:
picurls = Out[25]

In [ ]:
mediaRdd.flatMap(lambda x: posturlS3(x) ).collect()

In [ ]:
responses = Out[172]

In [ ]:
len(responses)

In [ ]:
set(responses)

In [11]:
import numpy as np

In [ ]:
responsearray= np.array(responses)

In [ ]:
whichnumber=np.where(responsearray==403)

In [ ]:
np.histogram(whichnumber,bins=100000).plot()

In [ ]:
print "Number of correct responses: ", sum(responsearray==200), "\n", "Number of redirects (307):",\
    sum(responsearray==307),"\nNumber of forbidden (403): ", sum(responsearray==403),\
    "\nNumber of page not found (404): ", sum(responsearray==404),\
    "\nNumber of service unavailable (503): ", sum(responsearray==503)

In [ ]:
posturllocal(mediaRdd.first())

In [3]:
#print the json in a pretty way
def pretty(d, indent=0):
   for key, value in d.iteritems():
      print '\t' * indent + str(key)
      if isinstance(value, dict):
         pretty(value, indent+1)
      #else:
      #   print '\t' * (indent+1) + str(value)
def getMediaUrl(obj):
    items = []
    for item in getNestedKeyValue(obj, ["media"]):
        items.append([obj['id'],getNestedKeyValue(item, ["media_url"])])
    return items

#find an item in the dictionary
def _finditem(obj, key):
    if key in obj: return obj[key]
    for k, v in obj.items():
        if isinstance(v,dict):
            item = _finditem(v, key)
            if item is not None:
                return item    

#return an item the 
def getNestedKeyValue(obj, keys):
    for num in range(len(keys)):
        if isinstance(obj, type({})):
            obj = _finditem(obj, keys[num])
        if obj is None:
            return ""
    return obj


In [ ]:
Out[13]

In [ ]:
getNestedKeyValue(Out[13], ["media"])(Out[13])

In [4]:
def getCoordinates(obj):
    x = getNestedKeyValue(obj, ["location", "geo", "coordinates"])
    if x == "" or x is None:
        return ""

    x = x[0]
    return [[(x[0][0] + x[2][0])/2, (x[0][1] + x[1][1])/2]]


In [28]:
import curl

In [29]:
import pycurl
import shutil

In [5]:
import pycurl
from io import BytesIO

def posturlS3():
    buffer = BytesIO()
    c = pycurl.Curl()
    c.setopt(c.URL,  u'http://pbs.twimg.com/media/B87lDs_CcAAZ06m.jpg')
    c.setopt(c.WRITEDATA, buffer)
    c.perform()
    c.close()
    
    

In [ ]:
pwd

In [6]:
import pycurl
from io import BytesIO

def posturllocal(tweetid_imgurlpair):
    localdir = 'twitter_images/'
    tweetid = tweetid_imgurlpair[0]
    imgurl = tweetid_imgurlpair[1]
    
    ext = "."+ imgurl.split('.')[-1]
    iduniq = tweetid.split(":")[-1]
    
    buffer = BytesIO()
    c = pycurl.Curl()
    c.setopt(c.URL,  imgurl)
    c.setopt(c.WRITEDATA, buffer)
    c.perform()
    c.close()
    
    f = open(localdir+iduniq+ext,'wb')
    f.write(buffer.getvalue())
    f.close()

In [10]:
import pycurl
from io import BytesIO
import boto

def posturlS3(tweetid_imgurlpair, aws_access_key_id=None, aws_secret_access_key=None):
    bucketid = 'pnnl-twitter-images'
    tweetid = tweetid_imgurlpair[0]
    imgurl = tweetid_imgurlpair[1]
    
    ext = "."+ imgurl.split('.')[-1]
    iduniq = tweetid.split(":")[-1]
    
    buffer = BytesIO()
    c = pycurl.Curl()
    c.setopt(c.URL,  imgurl)
    c.setopt(c.WRITEDATA, buffer)
    c.perform()
    #c.close()
    
    s3c = boto.connect_s3(aws_access_key_id, aws_secret_access_key)
    bucket = s3c.lookup(bucketid)
    key = bucket.key_class(b,iduniq+ext)
    key.set_contents_from_string(buffer.getvalue())
    return [c.getinfo(c.RESPONSE_CODE)]

In [ ]:
im = BytesIO()
c = pycurl.Curl()
c.setopt(c.URL,  u'http://pbs.twimg.com/media/B87lDs_CcAAZ06m.jpg')
c.setopt(c.WRITEDATA, im)
c.perform()
c.close()



In [ ]:
im.getvalue()

In [ ]:
len(im.getvalue())

In [9]:
import boto

In [ ]:
c = boto.connect_s3(aws_access_key_id, aws_secret_access_key)

In [ ]:
b = c.lookup('pnnl-twitter-images')

In [ ]:
k = b.key_class(b,'testimage6.jpg')

In [ ]:
k.send_file(open('/home/sander/B87lDs_CcAAZ06m.jpg','rb'))

In [ ]:
k.set_contents_from_filename('/home/sander/B87lDs_CcAAZ06m.jpg')

In [ ]:
results = b.list()

In [ ]:
results.bucket

In [ ]:
f = open('/home/sander/B87lDs_CcAAZ06m.jpg', 'rb')

In [ ]:
k = b.key_class(b,'testimage6.jpg')

In [ ]:
#k.size = len(im.getvalue())

In [ ]:
k.set_contents_from_string(im.getvalue())

In [ ]:
k.send_file(im)

In [9]:
testimageurl = r"http://s3.amazonaws.com/pnnl-twitter-images/634775225176064000.jpg"

In [5]:
buffer = BytesIO()
c = pycurl.Curl()
c.setopt(c.URL,  testimageurl)
c.setopt(c.WRITEDATA, buffer)
c.perform()
c.getinfo(c.RESPONSE_CODE)

NameError: name 'BytesIO' is not defined

In [8]:
import urllib

from StringIO import StringIO
import json


In [39]:
#function to query the GPU server for the classification category parse the JSON and add the tweet ID
def getHardhatClassificationJson(tweetid):
    c = pycurl.Curl()
    imageurl = "http://s3.amazonaws.com/pnnl-twitter-images/%s.jpg"%(tweetid)
    digitsurl = "http://10.25.48.216:8080/models/images/classification/classify_one.json"
    buf = StringIO()
    c.setopt(c.URL, digitsurl)
    c.setopt(c.POSTFIELDS, \
             urllib.urlencode((('job_id','20151030-223956-5b04'),('image_url',imageurl))))
    c.setopt(c.WRITEDATA, buf)
    c.setopt(c.POST, 1)
    c.perform()
    if c.getinfo(c.RESPONSE_CODE) == 200:
        jdict = json.loads(buf.getvalue())
        jdict['tweet_id'] = tweetid
        return [json.dumps(jdict)]
    else:
        return []

In [9]:
#function to query the GPU server for the classification category parse the JSON and add the tweet ID
def getClassificationJson(tweetid):
    c = pycurl.Curl()
    imageurl = "http://s3.amazonaws.com/pnnl-twitter-images/%s.jpg"%(tweetid)
    digitsurl = "http://10.25.48.216:8080/models/images/classification/classify_one.json"
    buf = StringIO()
    c.setopt(c.URL, digitsurl)
    c.setopt(c.POSTFIELDS, \
             urllib.urlencode((('job_id','20150910-211807-d169'),('image_url',imageurl))))
    c.setopt(c.WRITEDATA, buf)
    c.setopt(c.POST, 1)
    c.perform()
    if c.getinfo(c.RESPONSE_CODE) == 200:
        jdict = json.loads(buf.getvalue())
        jdict['tweet_id'] = tweetid
        return [json.dumps(jdict)]
    else:
        return []
    
    


In [37]:
#function to query the GPU server for the classification category parse the JSON and add the tweet ID
def getClassificationJson(tweetid):
    c = pycurl.Curl()
    imageurl = "http://s3.amazonaws.com/pnnl-twitter-images/%s.jpg"%(tweetid)
    digitsurl = "http://digirest-490437472.us-east-1.elb.amazonaws.com/models/images/classification/classify_one.json"
    buf = StringIO()
    c.setopt(c.URL, digitsurl)
    c.setopt(c.POSTFIELDS, \
             urllib.urlencode((('job_id','20150706-203919-1597'),('image_url',imageurl))))
    c.setopt(c.WRITEDATA, buf)
    c.setopt(c.POST, 1)
    c.perform()
    print "%s?%s" % (digitsurl,urllib.urlencode((('job_id','20150706-203919-1597'),('image_url',imageurl))))
    print c.getinfo(c.RESPONSE_CODE)
    if c.getinfo(c.RESPONSE_CODE) == 200:
        jdict = json.loads(buf.getvalue())
        jdict['tweet_id'] = tweetid
        return [json.dumps(jdict)]
    else:
        return []
    
    


In [29]:
jdict['tweet_id'] = testimageurl.split('/')[-1].split('.')[0]

NameError: name 'testimageurl' is not defined

In [40]:
getHardhatClassificationJson('642321840925503488')

['{"tweet_id": "642321840925503488", "predictions": [["hardhat", 92.81], ["not_hardhat", 7.19]]}']

In [36]:
getClassificationJson('642321840925503488')

http://digirest-490437472.us-east-1.elb.amazonaws.com/models/images/classification/classify_one?job_id=20150706-203919-1597&image_url=http%3A%2F%2Fs3.amazonaws.com%2Fpnnl-twitter-images%2F642321840925503488.jpg
500


[]

In [16]:
tweetIDMapRDD = jsondict.map(lambda x:  x['id'].split(":")[-1])

In [17]:
classifyJsonRDD = tweetIDMapRDD.flatMap(lambda x: getHardhatClassificationJson(x))

In [23]:
classifyJsonRDD = tweetIDMapRDD.flatMap(lambda x: getClassificationJson(x))

In [24]:
classifyJsonRDD.first()

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job 7 cancelled because Stage 7 was cancelled
	at org.apache.spark.scheduler.DAGScheduler.org$apache$spark$scheduler$DAGScheduler$$failJobAndIndependentStages(DAGScheduler.scala:1203)
	at org.apache.spark.scheduler.DAGScheduler.handleJobCancellation(DAGScheduler.scala:1173)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$handleStageCancellation$1.apply$mcVI$sp(DAGScheduler.scala:1161)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$handleStageCancellation$1.apply(DAGScheduler.scala:1160)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$handleStageCancellation$1.apply(DAGScheduler.scala:1160)
	at scala.collection.IndexedSeqOptimized$class.foreach(IndexedSeqOptimized.scala:33)
	at scala.collection.mutable.ArrayOps$ofInt.foreach(ArrayOps.scala:156)
	at org.apache.spark.scheduler.DAGScheduler.handleStageCancellation(DAGScheduler.scala:1160)
	at org

In [23]:
from pyspark.sql import SQLContext


In [24]:
sqlContext = SQLContext(sc)

In [ ]:
classifyJsonRDD.saveAsTextFile("tweetPredictions2.txt")

In [ ]:
print "hello"

In [ ]:
df = sqlContext.jsonRDD(classifyJsonRDD)

In [67]:
print "hello"

hello


In [68]:
df

DataFrame[predictions: array<array<string>>, tweet_id: string]

In [86]:
df.show()

Py4JJavaError: An error occurred while calling o223.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 24.0 failed 4 times, most recent failure: Lost task 0.3 in stage 24.0 (TID 72, c1-node-03.ec2.internal): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/opt/cloudera/parcels/CDH-5.4.5-1.cdh5.4.5.p0.7/jars/spark-assembly-1.3.0-cdh5.4.5-hadoop2.6.0-cdh5.4.5.jar/pyspark/worker.py", line 101, in main
    process()
  File "/opt/cloudera/parcels/CDH-5.4.5-1.cdh5.4.5.p0.7/jars/spark-assembly-1.3.0-cdh5.4.5-hadoop2.6.0-cdh5.4.5.jar/pyspark/worker.py", line 96, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/opt/cloudera/parcels/CDH-5.4.5-1.cdh5.4.5.p0.7/jars/spark-assembly-1.3.0-cdh5.4.5-hadoop2.6.0-cdh5.4.5.jar/pyspark/serializers.py", line 125, in dump_stream
    for obj in iterator:
  File "/opt/cloudera/parcels/CDH-5.4.5-1.cdh5.4.5.p0.7/lib/spark/python/pyspark/sql/c

In [71]:
df.printSchema()

root
 |-- predictions: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: string (containsNull = true)
 |-- tweet_id: string (nullable = true)



In [85]:
df.select("tweet_id","predictions","prediction.element[0].element[0]").show()

Py4JJavaError: An error occurred while calling o223.select.
: org.apache.spark.sql.AnalysisException: cannot resolve 'prediction.element[0].element[0]' given input columns predictions, tweet_id;
	at org.apache.spark.sql.catalyst.analysis.package$AnalysisErrorAt.failAnalysis(package.scala:42)
	at org.apache.spark.sql.catalyst.analysis.CheckAnalysis$$anonfun$apply$3$$anonfun$apply$1.applyOrElse(CheckAnalysis.scala:48)
	at org.apache.spark.sql.catalyst.analysis.CheckAnalysis$$anonfun$apply$3$$anonfun$apply$1.applyOrElse(CheckAnalysis.scala:45)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$transformUp$1.apply(TreeNode.scala:250)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$transformUp$1.apply(TreeNode.scala:250)
	at org.apache.spark.sql.catalyst.trees.CurrentOrigin$.withOrigin(TreeNode.scala:50)
	at org.apache.spark.sql.catalyst.trees.TreeNode.transformUp(TreeNode.scala:249)
	at org.apache.spark.sql.catalyst.plans.QueryPlan.org$apache$spark$sql$catalyst$plans$Query

In [77]:
df.first()

Py4JJavaError: An error occurred while calling o252.collect.
: org.apache.spark.SparkException: Job 16 cancelled because Stage 17 was cancelled
	at org.apache.spark.scheduler.DAGScheduler.org$apache$spark$scheduler$DAGScheduler$$failJobAndIndependentStages(DAGScheduler.scala:1203)
	at org.apache.spark.scheduler.DAGScheduler.handleJobCancellation(DAGScheduler.scala:1173)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$handleStageCancellation$1.apply$mcVI$sp(DAGScheduler.scala:1161)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$handleStageCancellation$1.apply(DAGScheduler.scala:1160)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$handleStageCancellation$1.apply(DAGScheduler.scala:1160)
	at scala.collection.IndexedSeqOptimized$class.foreach(IndexedSeqOptimized.scala:33)
	at scala.collection.mutable.ArrayOps$ofInt.foreach(ArrayOps.scala:156)
	at org.apache.spark.scheduler.DAGScheduler.handleStageCancellation(DAGScheduler.scala:1160)
	at org.apache.spark.scheduler.DAGSched

In [76]:
print "hello"

hello


In [ ]:
df.take(4)

In [ ]:
df.saveAsParquetFile("tweetpredictions2.parquet")

In [ ]:
ls

In [79]:
print "hello"

hello


In [ ]:
print "hello"